# Urban Sound Classification
8732 labeled sound excerpts of urban sounds from 10 classes

## <b>Context</b>
    
The automatic classification of environmental sound is a growing research field with multiple applications to largescale, content-based multimedia indexing and retrieval. In particular, the sonic analysis of urban environments is the subject of increased interest, partly enabled by multimedia sensor networks, as well as by large quantities of online multimedia content depicting urban scenes.

However, while there is a large body of research in related areas such as speech, music and bioacoustics, work on the analysis of urban acoustic environments is relatively scarce.Furthermore, when existent, it mostly focuses on the classification of auditory scene type, e.g. street, park, as opposed to the identification of sound sources in those scenes, e.g.car horn, engine idling, bird tweet.

There are primarily two major challenges with urban sound research namely

<ul><li>Lack of labeled audio data. Previous work has focused on audio from carefully produced movies or television tracks from specific environments such as elevators or office spaces and on commercial or proprietary datasets . The large effort involved in manually annotating real-world data means datasets based on field recordings tend to be relatively small (e.g. the event detection dataset of the IEEE AASP Challenge consists of 24 recordings per each of 17 classes).</li></ul>
<ul><li>Lack of common vocabulary when working on urban sounds.This means the classification of sounds into semantic groups may vary from study to study, making it hard to compare results so the objective of this notebook is to address the above two mentioned challenges.</li></ul>

## <b>Content</b>
The dataset is called UrbanSound and contains 8732 labeled sound excerpts (<=4s) of urban sounds from 10 classes: - The dataset contains 8732 sound excerpts (<=4s) of urban sounds from 10 classes, namely: Air Conditioner Car Horn Children Playing Dog bark Drilling Engine Idling Gun Shot Jackhammer Siren Street Music The attributes of data are as follows: ID – Unique ID of sound excerpt Class – type of sound

## <b>Acknowledgements</b>
Source of the dataset : https://drive.google.com/drive/folders/0By0bAi7hOBAFUHVXd1JCN3MwTEU

Source of research document : https://serv.cusp.nyu.edu/projects/urbansounddataset/salamon_urbansound_acmmm14.pdf

Source of Technique used : [CNN ARCHITECTURES FOR LARGE-SCALE AUDIO CLASSIFICATION](https://arxiv.org/pdf/1609.09430.pdf)

## Importing Basic Libraries and Installing Tools Required

In [1]:
%matplotlib inline
from memory_profiler import memory_usage
import os
import pandas as pd
from glob import glob
import numpy as np


### Installing libav-tools to get Librosa Working Properly

In [2]:
%%capture
!apt-get install libav-tools -y

### Importing Fast.ai for Deep Learning and Librosa for Creating Spectrogram

In [3]:
from fastai.vision import *
import librosa
import librosa.display
import pylab
import matplotlib
import gc

### Making Temporary Working Directories for Storing the Audio Conversions

In [4]:
!mkdir /kaggle/working/train
!mkdir /kaggle/working/test

## Defining the Create Spectrogram Function

In [5]:
def create_spectrogram(filename,name):
    plt.interactive(False)
    clip, sample_rate = librosa.load(filename, sr=None)
    fig = plt.figure(figsize=[0.72,0.72])
    ax = fig.add_subplot(111)
    ax.axes.get_xaxis().set_visible(False)
    ax.axes.get_yaxis().set_visible(False)
    ax.set_frame_on(False)
    S = librosa.feature.melspectrogram(y=clip, sr=sample_rate)
    librosa.display.specshow(librosa.power_to_db(S, ref=np.max))
    filename  = Path('/kaggle/working/train/' + name + '.jpg')
    plt.savefig(filename, dpi=400, bbox_inches='tight',pad_inches=0)
    plt.close()    
    fig.clf()
    plt.close(fig)
    plt.close('all')
    del filename,name,clip,sample_rate,fig,ax,S

In [6]:
def create_spectrogram_test(filename,name):
    plt.interactive(False)
    clip, sample_rate = librosa.load(filename, sr=None)
    fig = plt.figure(figsize=[0.72,0.72])
    ax = fig.add_subplot(111)
    ax.axes.get_xaxis().set_visible(False)
    ax.axes.get_yaxis().set_visible(False)
    ax.set_frame_on(False)
    S = librosa.feature.melspectrogram(y=clip, sr=sample_rate)
    librosa.display.specshow(librosa.power_to_db(S, ref=np.max))
    filename  = Path('/kaggle/working/test/' + name + '.jpg')
    plt.savefig(filename, dpi=400, bbox_inches='tight',pad_inches=0)
    plt.close()    
    fig.clf()
    plt.close(fig)
    plt.close('all')
    del filename,name,clip,sample_rate,fig,ax,S

### Splitting the Conversion in different cells and collecting the garbage to avoid Ram Overflow

In [7]:
Data_dir=np.array(glob("../input/train/Train/*"))

In [8]:
%load_ext memory_profiler

In [9]:
%%memit 
i=0
for file in Data_dir[i:i+1500]:
    filename,name = file,file.split('/')[-1].split('.')[0]
    create_spectrogram(filename,name)

peak memory: 409.26 MiB, increment: 36.33 MiB


In [10]:
gc.collect()

25196

In [11]:
%%memit 
i=1500
for file in Data_dir[i:i+1500]:
    filename,name = file,file.split('/')[-1].split('.')[0]
    create_spectrogram(filename,name)

peak memory: 409.65 MiB, increment: 11.08 MiB


In [12]:
gc.collect()

23794

In [13]:
%%memit 
i=3000
for file in Data_dir[i:i+1500]:
    filename,name = file,file.split('/')[-1].split('.')[0]
    create_spectrogram(filename,name)

peak memory: 409.80 MiB, increment: 14.37 MiB


In [14]:
gc.collect()

23798

In [15]:
%%memit 
i=4500
for file in Data_dir[i:]:
    filename,name = file,file.split('/')[-1].split('.')[0]
    create_spectrogram(filename,name)

peak memory: 409.70 MiB, increment: 10.61 MiB


In [16]:
gc.collect()

11249

## Creating Data Bunch for Training and Creating a Resnet34 Model

In [17]:
path = Path('/kaggle/working/')
np.random.seed(42)
data = ImageDataBunch.from_csv(path,csv_labels='../input/train.csv', folder="train", valid_pct=0.2, suffix='.jpg',
        ds_tfms=get_transforms(), size=224, num_workers=0).normalize(imagenet_stats)

In [18]:
data.classes

['air_conditioner',
 'car_horn',
 'children_playing',
 'dog_bark',
 'drilling',
 'engine_idling',
 'gun_shot',
 'jackhammer',
 'siren',
 'street_music']

In [19]:
learn = create_cnn(data, models.resnet34, metrics=accuracy)

Downloading: "https://download.pytorch.org/models/resnet34-333f7ec4.pth" to /tmp/.torch/models/resnet34-333f7ec4.pth
100%|██████████| 87306240/87306240 [00:01<00:00, 83971372.66it/s]


## Training The Model

In [20]:
learn.fit_one_cycle(4)

epoch,train_loss,valid_loss,accuracy
1,1.718776,0.926031,0.673413
2,1.136074,0.755203,0.726771
3,0.870916,0.600213,0.794848
4,0.746269,0.567910,0.816007


In [21]:
learn.save('stage-1')

In [22]:
learn.unfreeze()
learn.lr_find()
learn.recorder.plot()

LR Finder is complete, type {learner_name}.recorder.plot() to see the graph.


In [23]:
learn.fit_one_cycle(4, max_lr=slice(1e-4,1e-3))

epoch,train_loss,valid_loss,accuracy
1,0.561160,0.738423,0.791168
2,0.418634,0.301567,0.903404
3,0.245435,0.144982,0.959522
4,0.140169,0.134092,0.958602


In [24]:
learn.save('stage-2')

In [25]:
learn.lr_find()
learn.recorder.plot()

LR Finder is complete, type {learner_name}.recorder.plot() to see the graph.


In [26]:
learn.fit_one_cycle(1, max_lr=slice(1e-6,2e-6))

epoch,train_loss,valid_loss,accuracy
1,0.082762,0.129010,0.962282


In [27]:
learn.save('stage-3')

In [28]:
Test_dir=np.array(glob("../input/test/Test/*"))

In [29]:
%%memit 
i=0
for file in Test_dir[i:i+1500]:
    filename,name = file,file.split('/')[-1].split('.')[0]
    create_spectrogram_test(filename,name)

/opt/conda/lib/python3.6/site-packages/librosa/filters.py:284: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  warnings.warn('Empty filters detected in mel frequency basis. '
/opt/conda/lib/python3.6/site-packages/librosa/filters.py:284: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  warnings.warn('Empty filters detected in mel frequency basis. '
/opt/conda/lib/python3.6/site-packages/librosa/filters.py:284: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  warnings.warn('Empty filters detected in mel frequency basis. '
/opt/conda/lib/python3.6/site-packages/librosa/filters.py:284: UserWarning: Empty filters detected in mel 

peak memory: 1988.93 MiB, increment: 14.61 MiB


In [30]:
gc.collect()

22397

In [31]:
%%memit 
i=1500
for file in Test_dir[i:]:
    filename,name = file,file.split('/')[-1].split('.')[0]
    create_spectrogram_test(filename,name)

/opt/conda/lib/python3.6/site-packages/librosa/filters.py:284: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  warnings.warn('Empty filters detected in mel frequency basis. '
/opt/conda/lib/python3.6/site-packages/librosa/filters.py:284: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  warnings.warn('Empty filters detected in mel frequency basis. '
/opt/conda/lib/python3.6/site-packages/librosa/filters.py:284: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  warnings.warn('Empty filters detected in mel frequency basis. '
/opt/conda/lib/python3.6/site-packages/librosa/filters.py:284: UserWarning: Empty filters detected in mel 

peak memory: 1988.95 MiB, increment: 1.50 MiB


In [32]:
gc.collect()

14052

In [33]:
learn.load('stage-3')
test_csv = pd.read_csv('../input/test.csv')

## Making predictions and writing it to CSV

In [34]:
with open('output.csv',"w") as file:
    file.write("ID,Prediction\n")
    for test in test_csv.ID:
        img = open_image('/kaggle/working/test/'+str(test)+'.jpg')
        prediction = str(learn.predict(img)[0]).split()[0]
        file.write(str(test)+','+prediction)
        file.write('\n')

In [35]:
output = pd.read_csv('output.csv')
output.head()

,ID,Prediction
0,5,jackhammer
1,7,children_playing
2,8,drilling
3,9,dog_bark
4,13,street_music


### Removing Unwanted Folders

In [36]:
%%capture
!apt-get install zip
!zip -r train.zip /kaggle/working/train/
!zip -r test.zip /kaggle/working/test/
!rm -rf train/*
!rm -rf test/*